In [1]:
import langchain

In [2]:
langchain.__version__

'1.0.5'

In [3]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

True

## 1、测试

In [4]:
# 1.导入相关库
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
from langchain_community.tools.tavily_search import TavilySearchResults

# 2.导入模型和工具
web_search = TavilySearchResults(max_results=2)
model = ChatDeepSeek(model="deepseek-chat")

# 3.创建Agent
agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="你是一个智能助手，可以自己根据用户的问题来调用响应的工具帮助用户解决问题。"
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10032\1851043440.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search = TavilySearchResults(max_results=2)


In [5]:
# 4.运行Agent获得结果
result = agent.invoke(
    {"messages": [{"role": "user", "content": "请帮我查询LangChain1.0发布的时间和主要功能更新？"}]}
)

In [6]:
result

{'messages': [HumanMessage(content='请帮我查询LangChain1.0发布的时间和主要功能更新？', additional_kwargs={}, response_metadata={}, id='106bb7ef-f5ad-44ff-a436-77e1fb2fc696'),
  AIMessage(content='我来帮您查询LangChain 1.0的发布信息和主要功能更新。', additional_kwargs={'tool_calls': [{'id': 'call_00_8FVcBn05hN5eLqgItQJUd4xR', 'function': {'arguments': '{"query": "LangChain 1.0 release date time major features updates"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 216, 'total_tokens': 261, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 216}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'e499e3b4-948d-486f-bb99-80d3536cf86d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--53efaad9-5dc8-499e-92ba-5f8329

In [8]:
print(result['messages'][-1].content)

根据搜索结果，我为您整理了LangChain 1.0的发布信息：

## LangChain 1.0 发布时间

**发布日期：2025年10月22日**

这是LangChain的第一个稳定主要版本，标志着该框架从"实验性"向"生产就绪"的重要转变。

## 主要功能更新

### 1. **新的代理抽象 - createAgent**
- 引入了新的标准方式来构建LangChain代理
- 取代了LangGraph中的`createReactAgent`
- 提供更简洁、更强大的API

### 2. **中间件系统 (Middleware)**
- 全新的代理抽象层
- 解决了上下文工程控制的问题
- 使开发者能够更好地控制代理行为

### 3. **标准内容块 (Standard Content Blocks)**
- 新的`contentBlocks`属性
- 为所有提供商提供统一的现代LLM功能访问
- 改进了消息格式处理

### 4. **简化的包结构**
- 更简洁的框架组织
- 围绕三个核心改进进行优化

### 5. **生产就绪承诺**
- 承诺在2.0版本之前不会有破坏性变更
- 基于三年社区反馈的优化
- 专注于构建生产就绪的代理系统

### 6. **与LangGraph的深度集成**
- LangChain不再仅仅是"链"，而是以生产就绪代理为中心的平台
- 在LangGraph之上构建中央代理抽象

这个1.0版本的发布标志着LangChain从一个实验性框架向企业级生产就绪平台的重大转变。


In [9]:
len(result['messages'])

8

## 2、自定义工具

In [10]:
import requests,json

OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [11]:
from langchain.tools import tool
@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": os.getenv("OPENWEATHER_API_KEY"),    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [12]:
print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

get_weather
查询即时天气函数
:param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
:return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
{'loc': {'title': 'Loc'}}


In [14]:
# 创建Agent
agent = create_agent(
    model=model,
    tools=[get_weather,web_search],
    system_prompt="你是一个智能助手，可以自己根据用户的问题来调用响应的工具帮助用户解决问题。"
)


In [15]:
# 4.运行Agent获得结果
result = agent.invoke(
    {"messages": [{"role": "user", "content": "请问北京今天天气如何？"}]}
)

In [17]:
result['messages']

[HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}, id='9cc63ff0-ca84-49a8-95df-dab59d222b97'),
 AIMessage(content='我来帮您查询北京今天的天气情况。', additional_kwargs={'tool_calls': [{'id': 'call_00_twh0fgtW9zWRXA0YtLmqFawi', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 346, 'total_tokens': 369, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 26}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'db69fe9d-31f5-4484-ad62-793236c37296', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6c1102ec-b1e2-4a69-8ee0-ecbff5dbb5d1-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_00_twh0fgtW9zWRXA0YtLmq

In [16]:
result['messages'][-1].content

'根据查询结果，北京今天的天气情况如下：\n\n**天气状况**：阴，多云\n**温度**：14.94°C\n**体感温度**：13.17°C\n**湿度**：26%\n**气压**：1019 hPa\n**风速**：2.31 m/s\n**能见度**：10公里\n\n总体来说，今天北京天气阴凉，温度适中但湿度较低，风力较小。建议您根据这个温度适当穿着，由于湿度较低，可能需要注意保湿。'

In [18]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "请问今天杭州和上海哪里更冷？"}]}
)

In [20]:
result['messages']

[HumanMessage(content='请问今天杭州和上海哪里更冷？', additional_kwargs={}, response_metadata={}, id='24852e12-7689-4bc7-b4ee-48b4529d25d0'),
 AIMessage(content='我来帮您查询今天杭州和上海的天气情况，然后比较哪里更冷。', additional_kwargs={'tool_calls': [{'id': 'call_00_EDpuCMOWgamvBgDRFTaowYZg', 'function': {'arguments': '{"loc": "Hangzhou"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_01_s5AkWatYf95zqFns01FuTu6e', 'function': {'arguments': '{"loc": "Shanghai"}', 'name': 'get_weather'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 349, 'total_tokens': 395, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 29}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '523976a8-3b45-4883-8f5a-37c52a917235', 'service_tier': None, 'finish_reason': 'tool_calls', 'logpr

In [19]:
result['messages'][-1].content

'根据查询结果，我来为您比较今天杭州和上海的天气情况：\n\n**杭州天气：**\n- 温度：18.95°C\n- 体感温度：18.28°C\n- 天气状况：多云\n- 湿度：53%\n- 风速：3.42 m/s\n\n**上海天气：**\n- 温度：17.92°C\n- 体感温度：17.02°C\n- 天气状况：晴\n- 湿度：48%\n- 风速：4 m/s\n\n**比较结果：**\n今天**上海比杭州更冷**。上海的实际温度为17.92°C，体感温度为17.02°C；而杭州的实际温度为18.95°C，体感温度为18.28°C。上海的温度比杭州低了约1°C左右。\n\n虽然上海是晴天，杭州是多云天气，但上海的温度确实更低一些。'

## 多工具

In [21]:
import os
@tool
def write_to_markdown(content: str) -> str:
    """
    将指定内容保存为 Markdown 格式的文件。
    
    :param content: 要保存的文本内容。
    :param filename: 保存的文件名，默认为 'output.md'。
    :return: 返回保存结果的消息。
    """
    filename = "./output.md"
    try:
        # 确保文件名以 .md 结尾
        if not filename.endswith(".md"):
            filename += ".md"
        
        # 写入文件
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(content)
        
        return f"内容已成功保存为 {filename} 文件。"
    except Exception as e:
        return f"保存文件时发生错误：{e}"



In [22]:
# 创建Agent
agent_3 = create_agent(
    model=model,
    tools=[get_weather,web_search,write_to_markdown],
    system_prompt="你是一个智能助手，可以自己根据用户的问题来调用响应的工具帮助用户解决问题。"
)

In [23]:
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "请帮我查询北京、杭州、上海等地天气，并写入本地文件。"}]}
)

In [24]:
result['messages'][-1].content

'已完成！我已经为您查询了北京、杭州、上海三地的天气信息，并将详细报告保存到了本地文件 `output.md` 中。\n\n**主要天气情况总结：**\n- **北京**：阴天多云，温度约15°C，较为凉爽\n- **杭州**：多云天气，温度约19°C，湿度较高\n- **上海**：晴朗天气，温度约18°C，风速稍大\n\n所有详细信息都已保存到Markdown文件中，包括温度、湿度、风速、气压等完整气象数据。'

In [25]:
len(result['messages'])

8

In [26]:
result['messages']

[HumanMessage(content='请帮我查询北京、杭州、上海等地天气，并写入本地文件。', additional_kwargs={}, response_metadata={}, id='a8427e85-929b-43b7-9a08-ccd81bdeb218'),
 AIMessage(content='我来帮您查询北京、杭州、上海的天气信息，并保存到本地文件中。', additional_kwargs={'tool_calls': [{'id': 'call_00_gZGAGpoLkorhpTV5Ph4hXGYd', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_01_4tEtNLCeHyLG1PoXAzJqPPzK', 'function': {'arguments': '{"loc": "Hangzhou"}', 'name': 'get_weather'}, 'type': 'function', 'index': 1}, {'id': 'call_02_G810zBB3qWbRejpAfhx3afHt', 'function': {'arguments': '{"loc": "Shanghai"}', 'name': 'get_weather'}, 'type': 'function', 'index': 2}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 441, 'total_tokens': 500, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 249}, 'model_name': 'deepseek-chat', 's